![marvel](https://www.emp-online.es/dw/image/v2/BBQV_PRD/on/demandware.static/-/Library-Sites-EMPSharedLibrary/default/dwe169dcbd/100151_MT_Marvel.jpg?sw=350)

Para esta practica vamos a utilizar la api de [`Marvel`](developer.marvel.com)

Primero, necesitarás una clave API de Marvel, que puedes obtener registrándote en https://developer.marvel.com/

La practica consiste en crear un DataFrame de los personajes

Necesitamos que el DataFrame tenga la siguiente informacion:
1. id
2. nombre
3. url imagen
4. comics donde aparece
5. creador


## **Librerias**

In [30]:
import requests
import pandas as pd
import hashlib, datetime, json
from IPython.core.display import HTML

## **Crear DataFrame** atacando la `API`

In [36]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

def hash_params():
    """ Marvel API requires server side API calls to include
    md5 hash of timestamp + public key + private key """

    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

priv_key = "ca756b45569288aae3a1ab8e49484403d4f45c32"
pub_key = "e0fc8ad5189f126ce2b09c265befc3f0"
params = {"limit":100, 
          "nameStartsWith": "J", 
          'ts':timestamp, 
          'apikey':pub_key, 
          "hash": hash_params()}


url = 'http://gateway.marvel.com/v1/public/characters'
response = requests.get(url, params=params)
todos = json.loads(response.text)
persConJ = pd.DataFrame(todos["data"]["results"])


# Muestra 1 - Los comics en los que aparece cada personaje

In [66]:
heroe = pd.DataFrame()
for hero in range(len(persConJ)):
    id = persConJ.iloc[hero]["id"]
    nombre = persConJ.iloc[hero]["name"]
    imagen = persConJ.iloc[hero]["resourceURI"]
    comics0 = persConJ.iloc[hero]["comics"]["items"]

    newRecord = pd.DataFrame([[id, nombre, imagen]], columns=["Id", "Name", "Imagen"])

    for i in comics0:
        comic = i.get("name")
        aux = pd.DataFrame([comic], columns=["Comic"])
        conc = pd.concat([newRecord, aux], axis=1)
        heroe = pd.concat([heroe, conc])
    
heroe.set_index(["Id", "Name", "Comic"], inplace=True)

In [67]:
def to_img_tag(path):
    return '''<img src="' + path + '" width="50" >'''


display(HTML(heroe.to_html(escape=False, formatters=dict(Imagen=to_img_tag))))